In [30]:
import pandas as pd
import matplotlib.pyplot as plt


In [31]:
# Load Horas inicio fin centrífugas.xlsx
df = pd.read_excel('../../data/raw/Horas inicio fin centrífugas.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   EQUIPO     984 non-null    int64 
 1   Operación  984 non-null    object
 2   Orden      984 non-null    int64 
 3   DATEVALUE  984 non-null    object
dtypes: int64(2), object(2)
memory usage: 30.9+ KB


In [32]:
df.head()

,EQUIPO,Operación,Orden,DATEVALUE
0,14246,Centrifugació 1 ini,10005144,05-26-2023 13:45:41
1,14246,Centrifugació 1 fi,10005144,05-26-2023 15:26:57
2,14246,Centrifugació 2 ini,10005144,05-26-2023 16:56:30
3,14246,Centrifugació 2 fi,10005144,05-26-2023 18:18:12
4,14246,Centrifugació 1 ini,10005176,07-07-2023 11:53:24


In [33]:
# Duplicated rows
df.duplicated().sum()

np.int64(0)

## Equipo

In [34]:
# EQUIPO value counts
df['EQUIPO'].value_counts()

EQUIPO
14246    385
17825    306
12912    293
Name: count, dtype: int64

In [35]:
# To str
df['EQUIPO'] = df['EQUIPO'].astype(str)

## Operación

In [36]:
df['Operación'].value_counts()

Operación
Centrifugació 1 ini    247
Centrifugació 1 fi     247
Centrifugació 2 fi     246
Centrifugació 2 ini    244
Name: count, dtype: int64

In [37]:
# To str
df['Operación'] = df['Operación'].astype(str)
# For each centrifuga [1,2] Make new columns ini_1, ini_2,fin_1,fin_2
centrifugas = ['1', '2']
operaciones = ['ini', 'fi']

In [38]:
# Make column that matches centrifuga
df['Centrifuga'] = str(0)
for centrifuga in centrifugas:
    df.loc[df['Operación'] == 'Centrifugació ' + centrifuga + ' ini', 'Centrifuga'] = centrifuga
    df.loc[df['Operación'] == 'Centrifugació ' + centrifuga + ' fi', 'Centrifuga'] = centrifuga

In [39]:
# Make column that matches op
df['op'] = str(0)

for operacion in operaciones:
    df.loc[df['Operación'] == 'Centrifugació ' + '1 ' + operacion, 'op'] = operacion
    df.loc[df['Operación'] == 'Centrifugació ' + '2 ' + operacion, 'op'] = operacion
# Change ini and fi to 0 and 1
df['op'] = df['op'].str.replace('ini', '0')
df['op'] = df['op'].str.replace('fi', '1')

## Orden

In [40]:
# Value counts
df['Orden'].value_counts()

Orden
200209981    5
200200380    5
10005176     4
10005233     4
10005271     4
            ..
200209968    4
200209980    4
200179220    2
200182432    2
200184609    2
Name: count, Length: 247, dtype: int64

They should be in multiples of 2. Why is there one with 5

In [41]:
# To str
df['Orden'] = df['Orden'].astype(str)
# Check Orden 200209981
df[df['Orden'] == '200209981']

,EQUIPO,Operación,Orden,DATEVALUE,Centrifuga,op
372,14246,Centrifugació 1 ini,200209981,07-25-2024 07:27:48,1,0
373,14246,Centrifugació 1 fi,200209981,07-25-2024 07:43:45,1,1
374,14246,Centrifugació 2 ini,200209981,07-25-2024 11:17:35,2,0
375,14246,Centrifugació 2 fi,200209981,07-25-2024 12:04:23,2,1
376,14246,Centrifugació 2 fi,200209981,07-25-2024 13:46:48,2,1


We can probably assume its the later one. We should doublecheck anyways or drop this orden/value

In [42]:
# Check Orden 200200380
df[df['Orden'] == '200200380']

,EQUIPO,Operación,Orden,DATEVALUE,Centrifuga,op
855,12912,Centrifugació 1 ini,200200380,03-18-2024 08:40:15,1,0
856,12912,Centrifugació 1 fi,200200380,03-18-2024 12:12:21,1,1
857,12912,Centrifugació 2 ini,200200380,03-18-2024 12:44:51,2,0
858,12912,Centrifugació 2 fi,200200380,03-18-2024 12:59:37,2,1
859,12912,Centrifugació 2 fi,200200380,03-18-2024 14:44:06,2,1


Again.

In [43]:
# We will drop them for now
df = df[df['Orden'] != '200209981']
df = df[df['Orden'] != '200200380']
df['Orden'].value_counts()
# Reset index
df = df.reset_index(drop=True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974 entries, 0 to 973
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   EQUIPO      974 non-null    object
 1   Operación   974 non-null    object
 2   Orden       974 non-null    object
 3   DATEVALUE   974 non-null    object
 4   Centrifuga  974 non-null    object
 5   op          974 non-null    object
dtypes: object(6)
memory usage: 45.8+ KB


In [45]:
# Just to be sure
# Is there any Orden with more than 2 centrifugas?
df['Centrifuga'].value_counts()
#0 NULLs, which means its ok

Centrifuga
1    490
2    484
Name: count, dtype: int64

In [46]:
# If we group by Orden, centrifuga and op we should have 1 row per group
df.groupby(['Orden', 'Centrifuga', 'op']).size().value_counts()


1    974
Name: count, dtype: int64

## Reconvert dataframe

In [47]:
# We want to simplify the rows. We need 2 columns, one for inicio and one for end
df['Inicio'] = str(0)
df['Fin'] = str(0)
# Whenever we have a 0 in op, we want to put the time in inicio
df.loc[df['op'] == '0', 'Inicio'] = df['DATEVALUE']
# Whenever we have a 1 in op, we want to put the time in fin
df.loc[df['op'] == '1', 'Fin'] = df['DATEVALUE']
# Now we merge the rows
h = df.groupby(['EQUIPO', 'Orden', 'Centrifuga'], as_index=False).agg({'Inicio': 'first', 'Fin': 'last'})


In [48]:
df.head(20)

,EQUIPO,Operación,Orden,DATEVALUE,Centrifuga,op,Inicio,Fin
0,14246,Centrifugació 1 ini,10005144,05-26-2023 13:45:41,1,0,05-26-2023 13:45:41,0
1,14246,Centrifugació 1 fi,10005144,05-26-2023 15:26:57,1,1,0,05-26-2023 15:26:57
2,14246,Centrifugació 2 ini,10005144,05-26-2023 16:56:30,2,0,05-26-2023 16:56:30,0
3,14246,Centrifugació 2 fi,10005144,05-26-2023 18:18:12,2,1,0,05-26-2023 18:18:12
4,14246,Centrifugació 1 ini,10005176,07-07-2023 11:53:24,1,0,07-07-2023 11:53:24,0
5,14246,Centrifugació 1 fi,10005176,07-07-2023 15:06:24,1,1,0,07-07-2023 15:06:24
6,14246,Centrifugació 2 ini,10005176,07-07-2023 16:15:02,2,0,07-07-2023 16:15:02,0
7,14246,Centrifugació 2 fi,10005176,07-07-2023 17:58:02,2,1,0,07-07-2023 17:58:02
8,14246,Centrifugació 1 ini,10005183,09-08-2023 12:48:53,1,0,09-08-2023 12:48:53,0
9,14246,Centrifugació 1 fi,10005183,09-08-2023 16:18:56,1,1,0,09-08-2023 16:18:56


In [49]:
# Print orden == 10005144
print(h[h['Orden'] == 10005144])
h.head(2)


Empty DataFrame
Columns: [EQUIPO, Orden, Centrifuga, Inicio, Fin]
Index: []


,EQUIPO,Orden,Centrifuga,Inicio,Fin
0,12912,10005236,1,11-30-2023 08:38:34,11-30-2023 11:46:33
1,12912,10005236,2,11-30-2023 12:50:22,11-30-2023 14:27:29


In [50]:
h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   EQUIPO      487 non-null    object
 1   Orden       487 non-null    object
 2   Centrifuga  487 non-null    object
 3   Inicio      487 non-null    object
 4   Fin         487 non-null    object
dtypes: object(5)
memory usage: 19.2+ KB


In [51]:
h['EQUIPO'] = h['EQUIPO'].astype(str)
h['Orden'] = h['Orden'].astype(str)
h['Centrifuga'] = h['Centrifuga'].astype(str)
h['Inicio'] = pd.to_datetime(h['Inicio'])
h['Fin'] = pd.to_datetime(h['Fin'])

In [52]:
# Rename EQUIPO to Equipo
h = h.rename(columns={'EQUIPO': 'Equipo'})

## DATEVALUE

In [53]:
# DATEVALUE TO DATETIME
df['DATEVALUE'] = pd.to_datetime(df['DATEVALUE'])
df

,EQUIPO,Operación,Orden,DATEVALUE,Centrifuga,op,Inicio,Fin
0,14246,Centrifugació 1 ini,10005144,2023-05-26 13:45:41,1,0,05-26-2023 13:45:41,0
1,14246,Centrifugació 1 fi,10005144,2023-05-26 15:26:57,1,1,0,05-26-2023 15:26:57
2,14246,Centrifugació 2 ini,10005144,2023-05-26 16:56:30,2,0,05-26-2023 16:56:30,0
3,14246,Centrifugació 2 fi,10005144,2023-05-26 18:18:12,2,1,0,05-26-2023 18:18:12
4,14246,Centrifugació 1 ini,10005176,2023-07-07 11:53:24,1,0,07-07-2023 11:53:24,0
...,...,...,...,...,...,...,...,...
969,12912,Centrifugació 2 fi,200209978,2024-07-24 12:43:33,2,1,0,07-24-2024 12:43:33
970,12912,Centrifugació 1 ini,200209980,2024-07-25 07:44:43,1,0,07-25-2024 07:44:43,0
971,12912,Centrifugació 1 fi,200209980,2024-07-25 11:18:25,1,1,0,07-25-2024 11:18:25
972,12912,Centrifugació 2 ini,200209980,2024-07-25 12:08:37,2,0,07-25-2024 12:08:37,0


NEW POSSIBLE FEATURE: How many days between each centrifugation?

In [54]:
# is there any Inicio > Fin?
h[h['Inicio'] > h['Fin']]


,Equipo,Orden,Centrifuga,Inicio,Fin


In [55]:
# Is there any Inicio == Fin?
h[h['Inicio'] == h['Fin']]

,Equipo,Orden,Centrifuga,Inicio,Fin


## We should save h dataframe. NOT df as it is more useful.

In [56]:
h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Equipo      487 non-null    object        
 1   Orden       487 non-null    object        
 2   Centrifuga  487 non-null    object        
 3   Inicio      487 non-null    datetime64[ns]
 4   Fin         487 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 19.2+ KB


In [57]:
# Save to csv
h.to_csv('../../data/processed/horas.csv', index=False)